# Chemistry Assignment
The aim of the assignment is to validate the published densities’ data of elements in the periodic table by cross checking them with atomic data. Specifically, I am required to write a program that reads the atomic size of each element and its atomic weight, calculate the density of a single atom (atomic weight/ atomic volume) and compare the results with the tabulated density of the element (see second link). Note that each element has a theoretical and an empirical size: My calculation should be based on both.

I am further, required to calculate the distances between the atoms of a given element that would result in an accurate estimation of the tabulated densities. The results that I obtain should be compared with the published data for number of atoms per unit volume (see the second link).

Atomic radii data:
https://en.wikipedia.org/wiki/Atomic_radii_of_the_elements_(data_page)

Density data with number of atoms per volume:
https://en.wikipedia.org/wiki/Talk%3AList_of_elements_by_density/Numeric_densities

## Necessary imports for making the code work
I have taken the liberty to use the periodictable library for using the molar masses for each atom.

In [186]:
import periodictable
import math
import csv

## Reading in radii data

In [187]:
elementData = {}
with open("radiiData.csv") as f:
    data = csv.DictReader(f)
    for d in data:
        atomicNumber = int(d["atomic number"])
        elementData[atomicNumber] = {
            "symbol": d["symbol"],
            "name": d["name"],
            "empiricalRadius": int(d["empirical"]) if d["empirical"] != "no data" else None,
            "calculatedRadius": int(d["Calculated"]) if d["Calculated"] != "no data" else None,
            "molarMass": periodictable.elements[atomicNumber].mass
        }
f.close()

## Reading in density data with number of atoms per volume

In [188]:
with open("densityCountData.csv") as f:
    data = csv.DictReader(f)
    for d in data:
        atomicNumber = int(d["Atomic number"])
        if atomicNumber in elementData:
            elementData[atomicNumber]["density"] = float(d["Density"].split()[0]) if d["Density"] != "Unknown" else None
f.close()

## Calculating density of a single atom for empirical and calculated radius
Let's assume that the atom is formed as a sphere. Let $m = $ molar mass of the atom, $V = $ the volume and $r = $ the radius. Then the density $\rho$ of a single atom is
$$\rho=\frac{m}{V}=\frac{m}{\frac{4}{3}\pi r^3}=\frac{3m}{4\pi r^3} \left[\frac{\text{g}}{\text{pm}^3}\right]$$

In [189]:
density = lambda m, r : (3*m)/(4*math.pi*r**3)
with open("output.txt", "w+") as f:
    f.write("Density of a single atom for empirical and calculated radius:\n\n")
    for atomicNumber in elementData:
        f.write(f"{elementData[atomicNumber]['name']} ({elementData[atomicNumber]['symbol']}):\n")
        try:
            f.write(f"Density calculated with empirical radius: {density(elementData[atomicNumber]['molarMass'], elementData[atomicNumber]['empiricalRadius'])}\n")
        except:
            f.write("Could not calculate density\n")
        try:
            f.write(f"Density calculated with calculated radius: {density(elementData[atomicNumber]['molarMass'], elementData[atomicNumber]['calculatedRadius'])}\n\n")
        except:
            f.write("Could not calculate density\n\n")
f.close()

## Comparing the calculated densities with the tabulated density for each element

In [190]:
with open("output.txt", "a") as f:
    f.write("Comparing the calculated densities with the tabulated density for each element:\n\n")
    for atomicNumber in elementData:
        f.write(f"{elementData[atomicNumber]['name']} ({elementData[atomicNumber]['symbol']}):\n")
        try:
            f.write(f"Density error with empirical radius {abs(density(elementData[atomicNumber]['molarMass'], elementData[atomicNumber]['empiricalRadius'])-elementData[atomicNumber]['density'])}\n")
        except:
            f.write("Could not calculate error\n")
        try:
            f.write(f"Density error with empirical radius {abs(density(elementData[atomicNumber]['molarMass'], elementData[atomicNumber]['calculatedRadius'])-elementData[atomicNumber]['density'])}\n\n")
        except:
            f.write("Could not calculate error\n\n")
f.close()

## Calculating the distances between the atoms

Let's assume that the atoms are placed in a $1$ $\text{cm}^3$ cube. Then the total number of atoms are:
$$\text{Total number of atoms} = \frac{\text{Total mass}}{\text{Atomic mass}}=\frac{\rho}{\frac{m}{N_A}}=\frac{\rho\cdot N_A}{m}$$
The distance between the atoms is:
$$\text{distance}=\frac{\text{Length of one side of the cube}}{\sqrt[3]{\text{Total number of atoms}}}=\frac{1}{\sqrt[3]{\frac{\rho\cdot N_A}{m}}} \left[\text{cm}\right]=\frac{10^7}{\sqrt[3]{\frac{\rho\cdot N_A}{m}}} \left[\text{nm}\right]$$

In [191]:
AVOGADRO = periodictable.constants.avogadro_number
distance = lambda p, m : (10**7)/(((p*AVOGADRO)/(m))**(1/3))
with open("output.txt", "a") as f:
    f.write("Distance between atoms:\n\n")
    for atomicNumber in elementData:
        f.write(f"{elementData[atomicNumber]['name']} ({elementData[atomicNumber]['symbol']}):\n")
        try:
            f.write(f"Distance in nanometers: {distance(elementData[atomicNumber]['density'], elementData[atomicNumber]['molarMass'])}\n\n")
        except:
            f.write("Could not calculate distance\n\n")
f.close()

Look at output.txt for the answers for this assignment

## Using machine learning to predict density (just for fun)
The data is used to train and test a gradient boosted regressor. For now let's predict density given the other information

In [192]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import numpy as np

In [193]:
vars(periodictable.elements[1])

{'name': 'hydrogen',
 'symbol': 'H',
 'number': 1,
 '_isotopes': {2: H[2], 3: H[3], 1: H[1], 4: H[4], 5: H[5], 6: H[6]},
 'ions': (-1, 1),
 'ion': <periodictable.core.IonSet at 0x2309c5da888>,
 '_mass': 1.00794,
 '_density': 0.0708,
 'density_caveat': 'T=-252.87',
 'covalent_radius': 0.31,
 'covalent_radius_uncertainty': 0.05,
 'crystal_structure': {'symmetry': 'diatom', 'd': 0.74},
 'neutron': <periodictable.nsf.Neutron at 0x230a111c148>,
 '_xray': <periodictable.xsf.Xray at 0x230a1486708>}

In [194]:
df = pd.read_csv("data.csv")
df.replace(to_replace=["None"], value=np.nan, inplace=True)
for col in df:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.set_index("number")
df

,mass,density,density_caveat,covalent_radius,covalent_radius_uncertainty,crystal_structure
number,,,,,,
1,1.007940,0.0708,-252.87,0.31,0.05,0.74
2,4.002602,0.1220,-268.93,0.28,0.00,NaN
3,6.941000,0.5340,NaN,1.28,0.07,NaN
4,9.012182,1.8480,NaN,0.96,0.03,NaN
5,10.811000,2.3400,NaN,0.84,0.03,NaN
...,...,...,...,...,...,...
114,289.000000,NaN,NaN,NaN,NaN,NaN
115,289.000000,NaN,NaN,NaN,NaN,NaN
116,293.000000,NaN,NaN,NaN,NaN,NaN


In [195]:
# Using linear interpolation to fill nan values
lin = ["density", "density_caveat", "covalent_radius", "covalent_radius_uncertainty", "crystal_structure"]
df.loc[:, lin] = df.loc[:, lin].interpolate().bfill()
df

,mass,density,density_caveat,covalent_radius,covalent_radius_uncertainty,crystal_structure
number,,,,,,
1,1.007940,0.0708,-252.870,0.31,0.05,0.74
2,4.002602,0.1220,-268.930,0.28,0.00,0.80
3,6.941000,0.5340,-254.302,1.28,0.07,0.86
4,9.012182,1.8480,-239.674,0.96,0.03,0.92
5,10.811000,2.3400,-225.046,0.84,0.03,0.98
...,...,...,...,...,...,...
114,289.000000,14.0000,25.000,1.69,0.03,1.42
115,289.000000,14.0000,25.000,1.69,0.03,1.42
116,293.000000,14.0000,25.000,1.69,0.03,1.42


## Visualizing the data

In [196]:
fig = go.Figure()
for column in df.columns:
    fig.add_trace(go.Scatter(x=df.index, y=df[column], name=column))
fig.show()

In [197]:
# Preparing data for training
X = df.drop(columns=["density"])
y = df["density"]

X_train, X_test, y_train, y_test = train_test_split(X, y.values.flatten(), test_size=0.10, random_state=42, shuffle=False)

train_data = lgb.Dataset(X_train, label=y_train)
validation_data = lgb.Dataset(X_test, label=y_test)

In [180]:
# Creating gradient boosted regressor model
param = {'objective': 'regression_l2', "num_leaves": 200, "num_estimators": 100, "max_depth": 10, "max_bin": 100, "early_stopping_round": 30}
param['metric'] = 'l2'

# Train model
bst = lgb.train(param, train_data, 1000, valid_sets=[validation_data])

[1]	valid_0's l2: 23.8111
Training until validation scores don't improve for 30 rounds
[2]	valid_0's l2: 18.7157
[3]	valid_0's l2: 14.6538
[4]	valid_0's l2: 11.4224
[5]	valid_0's l2: 8.85784
[6]	valid_0's l2: 6.82813
[7]	valid_0's l2: 5.22689
[8]	valid_0's l2: 4.32391
[9]	valid_0's l2: 3.27727
[10]	valid_0's l2: 2.45911
[11]	valid_0's l2: 1.82304
[12]	valid_0's l2: 1.30144
[13]	valid_0's l2: 0.90698
[14]	valid_0's l2: 0.612693
[15]	valid_0's l2: 0.397026
[16]	valid_0's l2: 0.311667
[17]	valid_0's l2: 0.319516
[18]	valid_0's l2: 0.281305
[19]	valid_0's l2: 0.170344
[20]	valid_0's l2: 0.154532
[21]	valid_0's l2: 0.0864665
[22]	valid_0's l2: 0.112928
[23]	valid_0's l2: 0.139748
[24]	valid_0's l2: 0.0816466
[25]	valid_0's l2: 0.0788858
[26]	valid_0's l2: 0.106993
[27]	valid_0's l2: 0.0632291
[28]	valid_0's l2: 0.0847765
[29]	valid_0's l2: 0.0493023
[30]	valid_0's l2: 0.0745331
[31]	valid_0's l2: 0.073592
[32]	valid_0's l2: 0.0448254
[33]	valid_0's l2: 0.0662794
[34]	valid_0's l2: 0.0408856

In [198]:
# Testing the model
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(len(X_test))),
    y=bst.predict(X_test),
    # color=[i % 48 for i in range(len(X_test))],
    name='Predicted',
),
)

fig.add_trace(go.Scatter(
    x=list(range(len(y_test))),
    y=y_test,
    name='Actual',
    ))

fig.show()

As we can see, the model was not so far away from guessing the correct density, just a small difference of circa 0.1 :)